In [1]:
library(loadeR)

Loading required package: rJava

Loading required package: loadeR.java

Java version 11x amd64 by Oracle Corporation detected

NetCDF Java Library v4.6.0-SNAPSHOT (23 Apr 2015) loaded and ready

Loading required package: climate4R.UDG

climate4R.UDG version 0.2.3 (2021-07-05) is loaded


Get the latest stable version (0.2.4) using <devtools::install_github('SantanderMetGroup/climate4R.UDG')>

Please use 'citation("climate4R.UDG")' to cite this package.

loadeR version 1.7.1 (2021-07-05) is loaded

Please use 'citation("loadeR")' to cite this package.



In [2]:
dateset<-"esgeva/ensemble/CMIP6/CMIP/day/CMIP6_CMIP_AS-RCEC_TaiESM1_historical_day_gn_v20200626/replicas/esgf.ceda.ac.uk/CMIP6_CMIP_AS-RCEC_TaiESM1_historical_day_tas_gn_v20200626_esgf.ceda.ac.uk.ncml"

In [3]:
di<-dataInventory(url)

[2022-11-10 13:11:43] Doing inventory ...

[2022-11-10 13:11:43] Opening dataset...

[2022-11-10 13:11:44] The dataset was successfuly opened

[2022-11-10 13:11:45] Retrieving info for 'tas' (0 vars remaining)

[2022-11-10 13:11:45] Done.



In [4]:
di

$tas
$tas$Description
[1] "Near-Surface Air Temperature"

$tas$DataType
[1] "float"

$tas$Shape
[1]     1 60225   192   288

$tas$Units
[1] "K"

$tas$DataSizeMb
[1] 13320.81

$tas$Version
[1] NA

$tas$Dimensions
$tas$Dimensions$member
$tas$Dimensions$member$Type
[1] "Ensemble"

$tas$Dimensions$member$Units
[1] ""

$tas$Dimensions$member$Values
[1] "r1i1p1f1"


$tas$Dimensions$time
$tas$Dimensions$time$Type
[1] "Time"

$tas$Dimensions$time$TimeStep
[1] "1.0 days"

$tas$Dimensions$time$Units
[1] "days since 0001-01-01 00:00:00"

$tas$Dimensions$time$Date_range
[1] "1850-01-01T12:00:00Z - 2014-12-31T12:00:00Z"


$tas$Dimensions$lat
$tas$Dimensions$lat$Type
[1] "Lat"

$tas$Dimensions$lat$Units
[1] "degrees_north"

$tas$Dimensions$lat$Values
  [1] -90.0000000 -89.0575916 -88.1151832 -87.1727749 -86.2303665 -85.2879581
  [7] -84.3455497 -83.4031414 -82.4607330 -81.5183246 -80.5759162 -79.6335079
 [13] -78.6910995 -77.7486911 -76.8062827 -75.8638743 -74.9214660 -73.9790576
 [19] -73.0366492 -72.0942408 -71.1518325 -70.2094241 -69.2670157 -68.3246073
 [25] -67.3821990 -66.4397906 -65.4973822 -64.5549738 -63.6125654 -62.6701571
 [31] -61.7277487 -60.7853403 -59.8429319 -58.9005236 -57.9581152 -57.0157068
 [37] -56.0732984 -55.1308901 -54.1884817 -53.2460733 -52.3036649 -51.3612565
 [43] -50.4188482 -49.4764398 -48.5340314 -47.5916230 -46.6492147 -45.7068063
 [49] -44.7643979 -43.8219895 -42.8795812 -41.9371728 -40.9947644 -40.0523560
 [55] -39.1099476 -38.1675393 -37.2251309 -36.2827225 -35.3403141 -34.3979058
 [61] -33.4554974 -32.5130890 -31.5706806 -30.6282723 -29.6858639 -28.7434555
 [67] -27.8010471 -26.8586387 -25.9162304 -24.9738220 -24.0314136 -23.0890052
 [73] -22.1465969 -21.2041885 -20.2617801 -19.3193717 -18.3769634 -17.4345550
 [79] -16.4921466 -15.5497382 -14.6073298 -13.6649215 -12.7225131 -11.7801047
 [85] -10.8376963  -9.8952880  -8.9528796  -8.0104712  -7.0680628  -6.1256545
 [91]  -5.1832461  -4.2408377  -3.2984293  -2.3560209  -1.4136126  -0.4712042
 [97]   0.4712042   1.4136126   2.3560209   3.2984293   4.2408377   5.1832461
[103]   6.1256545   7.0680628   8.0104712   8.9528796   9.8952880  10.8376963
[109]  11.7801047  12.7225131  13.6649215  14.6073298  15.5497382  16.4921466
[115]  17.4345550  18.3769634  19.3193717  20.2617801  21.2041885  22.1465969
[121]  23.0890052  24.0314136  24.9738220  25.9162304  26.8586387  27.8010471
[127]  28.7434555  29.6858639  30.6282723  31.5706806  32.5130890  33.4554974
[133]  34.3979058  35.3403141  36.2827225  37.2251309  38.1675393  39.1099476
[139]  40.0523560  40.9947644  41.9371728  42.8795812  43.8219895  44.7643979
[145]  45.7068063  46.6492147  47.5916230  48.5340314  49.4764398  50.4188482
[151]  51.3612565  52.3036649  53.2460733  54.1884817  55.1308901  56.0732984
[157]  57.0157068  57.9581152  58.9005236  59.8429319  60.7853403  61.7277487
[163]  62.6701571  63.6125654  64.5549738  65.4973822  66.4397906  67.3821990
[169]  68.3246073  69.2670157  70.2094241  71.1518325  72.0942408  73.0366492
[175]  73.9790576  74.9214660  75.8638743  76.8062827  77.7486911  78.6910995
[181]  79.6335079  80.5759162  81.5183246  82.4607330  83.4031414  84.3455497
[187]  85.2879581  86.2303665  87.1727749  88.1151832  89.0575916  90.0000000

$tas$Dimensions$lat$Shape
[1] 192

$tas$Dimensions$lat$Coordinates
[1] "lat"


$tas$Dimensions$lon
$tas$Dimensions$lon$Type
[1] "Lon"

$tas$Dimensions$lon$Units
[1] "degrees_east"

$tas$Dimensions$lon$Values
  [1]   0.00   1.25   2.50   3.75   5.00   6.25   7.50   8.75  10.00  11.25
 [11]  12.50  13.75  15.00  16.25  17.50  18.75  20.00  21.25  22.50  23.75
 [21]  25.00  26.25  27.50  28.75  30.00  31.25  32.50  33.75  35.00  36.25
 [31]  37.50  38.75  40.00  41.25  42.50  43.75  45.00  46.25  47.50  48.75
 [41]  50.00  51.25  52.50  53.75  55.00  56.25  57.50  58.75  60.00  61.25
 [51]  62.50  63.75  65.00  66.25  67.50  68.75  70.00  71.25  72.50  73.75
 [61]  75.00  76.25  77.50  78.75  80.00  81.25  82.50  83.75  85.00  86.25
 [71]  87.50  88

In [5]:
data<-loadGridData(url, var="tas", years=2010)

[2022-11-10 13:13:13] Opening dataset...

[2022-11-10 13:13:14] The dataset was successfuly opened

[2022-11-10 13:13:14] Defining geo-location parameters

[2022-11-10 13:13:14] Defining time selection parameters

[2022-11-10 13:13:15] Retrieving data subset ...



ERROR: Error in .jcall("RJavaTools", "Ljava/lang/Object;", "invokeMethod", cl, : java.io.IOException: java.io.IOException: java.io.EOFException; tas -- 0:0,58400:58764,0:191,145:287


In [ ]:
data